In [1]:
import praw
r = praw.Reddit(user_agent='/r/Analog_Scraper by /u/compassdestroyer')
submissions = r.get_subreddit('opensource').get_hot(limit=5)
[str(x) for x in submissions]

['0 :: Community Survy',
 '60 :: Flarum \xe2\x80\x93 Free, open-source forum software with a focus on simplicity.',
 '0 :: Platform cooperativism as a critique of open-source',
 '1 :: Open source software communities?',
 '8 :: Guy goes to Eurovision, releases hundreds of freely licensed images']

In [4]:
user_agent = "/r/Analog_Scraper by /u/compassdestroyer"
r = praw.Reddit(user_agent=user_agent)

In [13]:
#items.py
from scrapy import Item, Field

class RedditItem(Item):
    subreddit = Field()
    link = Field()
    title = Field()
    authors = Field()
    date = Field()
    vote = Field()
    #top_comment = Field()

In [17]:
import re

from bs4 import BeautifulSoup

from scrapy import Spider, Request
from items import RedditItem

In [18]:
class RedditSpider(Spider):
    name = 'reddit'
    allowed_domains = ['reddit.com']
    start_urls = ['https://www.reddit.com/r/circlejerk',
                  'https://www.reddit.com/r/gaming',
                  'https://www.reddit.com/r/floridaman',
                  'https://www.reddit.com/r/movies',
                  'https://www.reddit.com/r/science',
                  'https://www.reddit.com/r/seahawks',
                  'https://www.reddit.com/r/totallynotrobots',
                  'https://www.reddit.com/r/uwotm8',
                  'https://www.reddit.com/r/videos',
                  'https://www.reddit.com/r/worldnews']

In [19]:
def parse(self, response): 
    links = response.xpath('//p[@class="title"]/a[@class="title may-blank "]/@href').extract()
    titles = response.xpath('//p[@class="title"]/a[@class="title may-blank "]/text()').extract()
    dates = response.xpath('//p[@class="tagline"]/time[@class="live-timestamp"]/@title').extract()
    authors = response.xpath('//p[@class="tagline"]/a[@class="author may-blank "]/text').extract()
    votes = response.xpath('//div[@class="midcol unvoted"]/div[@class="score unvoted"]/text()').extract()
    #comments = response.xpath('//div[@id="siteTable"]//a[@class="comments may-blank"]/@href').extract()

In [20]:
#settings.py
BOT_NAME = 'reddit'
SPIDER_MODULES = ['reddit.spiders']
NEWSPIDER_MODULE = 'reddit.spiders'

DOWNLOAD_DELAY = 2

ITEM_PIPELINES = {'reddit.pipelines.DuplicatesPipeline':300, 'reddit.pipelines.MongoDBPipeline':800, }

MONGODB_SERVER = "localhost"
MONGODB_PORT = 27017
MONGODB_DB = "reddit"
MONGODB_COLLECTION = "post" 

In [22]:
import pymongo 
from scrapy.conf import settings 
from scrapy.exceptions import DropItem 
from scrapy import log 

class DuplicatesPipeline(object):
    
    def __init__(self): 
        self.ids_seen = set()
        
    def process_item(self, item, spider):
        if item['link'] in self.ids_seen:
            raise DropItem("Duplicate item found: %s" % item) 
        else: 
            self.ids_seen.add(item['link']) 
            return item 
class MongoDBPipeline(object):
    def __init__(self): 
        connection = pymongo.MongoClient(
            settings['MONGODB_SERVER'],
            settings['MONGODB_PORT'] 
        ) 
        db = connection[settings['MONGODB_DB']]
        self.collection = db[settings['MONGODB_COLLECTION']]
        
    def process_item(self, item, spider):
        valid = True 
        for data in item:
            if not data: 
                valid = False
                raise DropItem("Missing {0}!".format(data))
        if valid: 
            self.collection.insert(dict(item))
            log.msg("Added to MongoDB database!",
                    level=log.DEBUG, spider=spider) 
            return item

/Users/zlandes/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: ScrapyDeprecationWarning: Module `scrapy.conf` is deprecated, use `crawler.settings` attribute instead
  from ipykernel import kernelapp as app
/Users/zlandes/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: ScrapyDeprecationWarning: Module `scrapy.log` has been deprecated, Scrapy now relies on the builtin Python library for logging. Read the updated logging entry in the documentation to learn more.
